# AV: Bigmart Sales Prediction - Solution 6

In [126]:
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

In [127]:
data = pd.read_csv('Train_UWu5bXk.csv')
test = pd.read_csv("Test_u94Q5KV.csv")

In [128]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [129]:
features = ['Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type',
       'Outlet_Type']
target = ['Item_Outlet_Sales']

In [130]:
item_identifier_weight = data[['Item_Identifier', 'Item_Weight']]

unique_item_identifier = data['Item_Identifier'].unique()
item_identifier_weight_dic = {x: np.NaN for x in unique_item_identifier}

for j in item_identifier_weight.iterrows():
        if not np.isnan(j[1]['Item_Weight']):
            item_identifier_weight_dic[j[1]['Item_Identifier']] = j[1]['Item_Weight']
            
def f(x):
    a = x[0]
    b = x[1]
    if np.isnan(x[1]):
        b = item_identifier_weight_dic[x[0]]
    return pd.Series([a, b], ['Item_Identifier', 'Item_Weight'])
           
item_identifier_weight_data = item_identifier_weight.apply(f, axis=1)

In [131]:
item_identifier_weight = test[['Item_Identifier', 'Item_Weight']]

#unique_item_identifier = test['Item_Identifier'].unique()
#item_identifier_weight_dic = {x: np.NaN for x in unique_item_identifier}

for j in item_identifier_weight.iterrows():
        if not np.isnan(j[1]['Item_Weight']):
            item_identifier_weight_dic[j[1]['Item_Identifier']] = j[1]['Item_Weight']
            
def f(x):
    a = x[0]
    b = x[1]
    if np.isnan(x[1]):
        b = item_identifier_weight_dic[x[0]]
    return pd.Series([a, b], ['Item_Identifier', 'Item_Weight'])
           
item_identifier_weight = item_identifier_weight.apply(f, axis=1)

In [132]:
data = data[['Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales']]
data = pd.concat([data, item_identifier_weight_data], axis=1)
data = data[data['Item_Weight'] >= 0]
data.head()

,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Identifier,Item_Weight
0,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380,FDA15,9.30
1,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228,DRC01,5.92
2,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700,FDN15,17.50
3,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800,FDX07,19.20
4,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052,NCD19,8.93


In [133]:
test = test[['Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Location_Type',
       'Outlet_Type']]
test = pd.concat([test, item_identifier_weight_test], axis=1)
#test = test[test['Item_Weight'] >= 0]
test.head()

,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Identifier,Item_Weight
0,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Tier 1,Supermarket Type1,FDW58,20.750
1,reg,0.038428,Dairy,87.3198,OUT017,2007,Tier 2,Supermarket Type1,FDW14,8.300
2,Low Fat,0.099575,Others,241.7538,OUT010,1998,Tier 3,Grocery Store,NCN55,14.600
3,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Tier 2,Supermarket Type1,FDQ58,7.315
4,Regular,0.118599,Dairy,234.2300,OUT027,1985,Tier 3,Supermarket Type3,FDY38,13.600


In [134]:
test.isnull().sum()

Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Identifier              0
Item_Weight                  0
dtype: int64

In [135]:
print(data.columns)
X = data[features]
y = data['Item_Outlet_Sales']
X_test = test[features]

Index(['Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP',
       'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales',
       'Item_Identifier', 'Item_Weight'],
      dtype='object')


## Imputing missing values
Note: features with null values has not been taken in the first round

In [136]:
X.isnull().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [137]:
X_test.isnull().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [138]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)
X.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,1999,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,5.92,0.019278,48.2692,2009,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,17.50,0.016760,141.6180,1999,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,19.20,0.000000,182.0950,1998,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,8.93,0.000000,53.8614,1987,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [139]:
X_test.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,20.750,0.007565,107.8622,1999,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,8.300,0.038428,87.3198,2007,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,14.600,0.099575,241.7538,1998,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,7.315,0.015388,155.0340,2007,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,13.600,0.118599,234.2300,1985,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## Scaling the data

In [140]:
sc = StandardScaler()
X = sc.fit_transform(X)
X_test = sc.transform(X_test)

## Dividing the data in training and validation set

In [141]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

## Training

In [142]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

## Predictions

In [143]:
predictions = model_lr.predict(X_val)
print(model_lr.score(X_val, y_val))
model_lr.score(X_train, y_train)

0.562635039889


0.56322957065655022

## Writing the submission file

In [144]:
model_lr = LinearRegression()
model_lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [145]:
predictions = model_lr.predict(X_test)

In [146]:
result = test[['Item_Identifier', 'Outlet_Identifier']]
result['Item_Outlet_Sales'] = pd.Series(predictions, index=result.index)
result.to_csv('submission.csv', index = False, index_label = 'Id' )

C:\Users\ranjank\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
